# Example 2

It's an example with sparse matrix $B$ $\left(f(x) = \overline{f}(Bx)\right)$ showing that PCA will produce a dense response while sPCA will give a sparse one

In [1]:
import sys
import os.path as op
sys.path = [op.abspath('../')] + sys.path + [op.abspath('../')]

In [2]:
import regression
import seaborn as sns
sns.set()
%matplotlib inline

In [3]:
import numpy as np
import scipy
from scipy.linalg import sqrtm, inv

In [57]:
def sym(w):
    return w.dot(inv(sqrtm(w.T.dot(w))))

def get_data(sample_size=500, noise_std=0.03):
    # generate covariance mat
    U = np.random.rand(8, 8)
    U = sym(U)
    S = np.diag([2, 1.7, 1.4, 1.1, 0.9, 0.7, 0.4, 0.1])
    cov = np.dot(np.dot(U, S), U.T)
    # generate centered inputs
    X = np.random.multivariate_normal(np.zeros((1, 8))[0], cov, sample_size)
    X -= X.mean(0)
    B = scipy.sparse.rand(X.shape[1], 2, density=0.2, random_state=7)
    y = func(X, B.A) + noise_std * np.random.randn(sample_size)
    return X, y, B.A

def func(X, B):
    return  np.sum(np.tanh(np.dot(X, B)),axis = 1)

In [58]:
X, y, B = get_data()

In [62]:
from sklearn.decomposition import PCA, SparsePCA
from edrgp.regression import GaussianProcessRegressor
from edrgp.edr import EffectiveDimensionalityReduction
PCA_edr = EffectiveDimensionalityReduction(GaussianProcessRegressor(),
                                           PCA(n_components=2), normalize=True)
PCA_edr.fit(X, y)

In [69]:
PCA_edr.components_.T

array([[ 0.00400796, -0.00813436],
       [-0.14618737, -0.76992173],
       [ 0.00505738, -0.05096732],
       [ 0.00436743, -0.00995607],
       [-1.16176695,  0.20762962],
       [ 0.01971241, -0.0355548 ],
       [-0.03731789,  0.06120155],
       [-0.09383576, -0.54754552]])

In [73]:
sPCA_edr = EffectiveDimensionalityReduction(GaussianProcessRegressor(),
                                            SparsePCA(n_components=2),
                                            normalize=True)
sPCA_edr.fit(X, y)

In [82]:
sPCA_edr.components_.T/14.73

array([[ 0.        ,  0.        ],
       [ 0.        ,  0.49916413],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [-0.97059779,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.        ],
       [ 0.        ,  0.32989411]])

In [83]:
sPCA_edr.scaler

AttributeError: 'EffectiveDimensionalityReduction' object has no attribute 'scaler_'

In [81]:
14.29/0.97

14.731958762886597